# Practico 3 y 4 (parte 6)

## Ablación de datos

A medida que los sistemas de aprendizaje automático se utilizan en un número cada vez mayor de aplicaciones, desde el análisis de datos sensoriales de satélites y el análisis de la atención de la salud hasta los asistentes virtuales inteligentes y los coches autoconductores, también se están volviendo cada vez más complejos. Esto significa que se necesita más tiempo y recursos de computación para entrenar los modelos y el número de opciones de diseño e hiperparámetros también aumentará. Debido a esta complejidad, suele ser difícil explicar el efecto de cada elección de diseño o componente del sistema de aprendizaje automático sobre su rendimiento. En el contexto del aprendizaje por máquina, podemos definir el estudio de la ablación como "un examen científico de un sistema de aprendizaje por máquina mediante la eliminación de sus componentes básicos para comprender sus efectos sobre su rendimiento general". Las características del conjunto de datos y los componentes del modelo son ejemplos notables de estos bloques de construcción (por lo tanto, utilizamos sus correspondientes términos de ablación de características y ablación de modelos), pero cualquier elección de diseño o módulo del sistema puede incluirse en un estudio de ablación. Sin embargo, los estudios de ablación no forman parte actualmente de la práctica estándar de aprendizaje de máquinas. Una de las razones fundamentales de ello es el hecho de que actualmente la realización de un estudio de ablación requiere importantes modificaciones del código, así como recursos informáticos y de tiempo adicionales. Por otra parte, la experimentación con un sistema de aprendizaje automático es un proceso iterativo que consiste en varios ensayos.

* [Ablation Programming for Machine Learning](http://www.diva-portal.org/smash/record.jsf?aq2=%5B%5B%5D%5D&c=1&af=%5B%5D&searchType=SIMPLE&sortOrder2=title_sort_asc&query=sina%2Bsheikholeslami&language=en&pid=diva2%3A1349978&aq=%5B%5B%5D%5D&sf=all&aqe=%5B%5D&sortOrder=author_sort_asc&onlyFullText=false&noOfRows=50&dswid=-716)
* [In the context of deep learning, what is an ablation study?](https://www.quora.com/In-the-context-of-deep-learning-what-is-an-ablation-study)

## Importacion de modulos y librerias

In [1]:
import os
import numpy as np
import pandas as pd

from ast import literal_eval

from utiles import student_rating_category
from utiles import ablation

data_dir = os.path.join('..', 'dataset')

## Procesamiento del data frame de metadata `md_df`

1. Se carga el conjunto de datos correspondiente a la metadata
2. Se conservan las columnas `session_id` y `student_rating` con el objetivo de convertir la variable `student_rating en una variable categórica para indicar el éxito o fracaso de sesion.
3. Se eliminan del data frame las filas cuya entrada en la columna `student_rating` no contengan datos.
4. Se convierte a variable categórica la variable `student_rating` mediante el uso de la funcion auxiliar `student_rating_category`.
5. Se remueven las filas cuya entrada en la columna `student_rating` contenga la plabra 'neutra'. De este modo la variable categorica es binaria, indicando con un 1 éxito y con un 0 fracaso.
6. Se imprime información de estado del data frame.

Finalmente, obtendremos un data frame con la variable categórica binaria `student_rating` y su correspondiente número de sesión `session_id` que permitira asociar el resultado de la sesión al conjunto de datos correspondiente a los diálogos. 

In [2]:
#1. Dataset de metadata
md_data_file_name = 'datadump-20150801-20171219.csv'
md_full_data_file_name = os.path.join(data_dir, md_data_file_name)
md_df = pd.read_csv(md_full_data_file_name, low_memory=False)

#2. Selección de columnas
md_df.columns = [column.strip() for column in md_df.columns]
md_df = md_df[['session_id','student_rating']]

#3. Eliminamos las filas nulas
md_df = md_df.dropna(subset=['student_rating'])

#4. Categorización de la variable numérica
md_df['student_rating'] = md_df.student_rating.apply(student_rating_category)

#5. Remoción de las filas con calificación neutra
md_df = md_df.loc[md_df.student_rating != 'neutra'].astype(int)

#6. Información del data frame resultante con la categorizacion de las sesiones
md_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46164 entries, 0 to 63264
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   session_id      46164 non-null  int64
 1   student_rating  46164 non-null  int64
dtypes: int64(2)
memory usage: 1.1 MB


## Carga de los conjunto de datos de mensaje

1. Se carga el conjunto de datos correspondiente a los mensajes entre tutor y estudiante
2. Conservamos las columnas que son necesarias para el entrenamiento del modelo. En este caso `text` cotiene el texto de interés, `sent_from` identifica el originante del mensaje a los fines de poder analizar su impacto mediante ablación de datos y `session_id` permitira realizar la composición de ambos datasets.
3. Se remueven todas las filas que no contengan mensajes del estudiante o tutor.
4. Se convierte la representacion textual de las listas en listas de python propiamente.
5. Se imprime información de estado del data frame.

In [3]:
#1. Dataset de mensajes
msg_file_name = 'yup_messages_preprocessed.csv'
msg_full_file_name = os.path.join(data_dir, msg_file_name)
msg_df = pd.read_csv(msg_full_file_name)

#2. Selección de columnas
msg_df = msg_df[['session_id', 'sent_from', 'text']]

#3. Tomamos solo las filas con contenido del tutor o el estudiante
msg_df = msg_df[msg_df.sent_from.isin(['student','tutor'])]

#4. Convertimos la representacion en str de las listas en listas propiamente
msg_df['text'] = msg_df.text.apply(lambda x: literal_eval(x))

#5. Información de estado del data frame
msg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1411477 entries, 0 to 1574068
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   session_id  1411477 non-null  int64 
 1   sent_from   1411477 non-null  object
 2   text        1411477 non-null  object
dtypes: int64(1), object(2)
memory usage: 43.1+ MB


## Casos de ablación

Tomaremos el 20%, 40%, 60%, 80% y el 100% de los turnos iniciando desde el final de la conversación.

In [4]:
tokeep_list = [0.2, 0.4 , 0.6, 0.8, 1.0]

### Estudiantes y tutores

In [5]:
for tokeep in tokeep_list:
    df = ablation(df=msg_df, tokeep=tokeep)
    df = df.groupby('session_id').sum()
    df = df.drop(columns='sent_from')
    df = df.reset_index()
    df = pd.merge(df, md_df, how='left', on=['session_id'])
    df = df.dropna(subset=['student_rating'])
    df['student_rating'] = df.student_rating.astype(int)

    pathdf = os.path.join(data_dir, 'ablations', f'yup_messages_ablated_{tokeep:05.2f}_students_tutors.csv')
    df.to_csv(pathdf, index=False)

### Estudiantes

In [6]:
for tokeep in tokeep_list:
    df = ablation(df=msg_df[msg_df.sent_from=='student'], tokeep=tokeep)
    df = df.groupby('session_id').sum()
    df = df.drop(columns='sent_from')
    df = df.reset_index()
    df = pd.merge(df, md_df, how='left', on=['session_id'])
    df = df.dropna(subset=['student_rating'])
    df['student_rating'] = df.student_rating.astype(int)

    pathdf = os.path.join(data_dir, 'ablations', f'yup_messages_ablated_{tokeep:05.2f}_students.csv')
    df.to_csv(pathdf, index=False)

### Tutores

In [7]:
for tokeep in tokeep_list:
    df = ablation(df=msg_df[msg_df.sent_from=='tutor'], tokeep=tokeep)
    df = df.groupby('session_id').sum()
    df = df.drop(columns='sent_from')
    df = df.reset_index()
    df = pd.merge(df, md_df, how='left', on=['session_id'])
    df = df.dropna(subset=['student_rating'])
    df['student_rating'] = df.student_rating.astype(int)

    pathdf = os.path.join(data_dir, 'ablations', f'yup_messages_ablated_{tokeep:05.2f}_tutors.csv')
    df.to_csv(pathdf, index=False)